# DSCI 551 Project

Date range: 2020-12-02 to 2022-03-29

### All necessary imports

In [3]:
import json
import numpy as np
import pandas as pd
import requests
import firebase_admin
from firebase_admin import db
from firebase_admin import credentials
import json

### Import and preview vaccination data

In [46]:
vacc_df = pd.read_csv('./data/country_vaccinations.csv')
vacc_df['date'] = pd.to_datetime(vacc_df['date'])
vacc_df['year'] = vacc_df.apply(lambda row: row.date.year, axis=1)
vacc_df['month'] = vacc_df.apply(lambda row: row.date.month, axis=1)
vacc_df['day'] = vacc_df.apply(lambda row: row.date.day, axis=1)
vacc_df.head(3)

,country,iso_code,date,total_vaccinations,people_vaccinated,people_fully_vaccinated,daily_vaccinations_raw,daily_vaccinations,total_vaccinations_per_hundred,people_vaccinated_per_hundred,people_fully_vaccinated_per_hundred,daily_vaccinations_per_million,vaccines,source_name,source_website,year,month,day
0,Afghanistan,AFG,2021-02-22,0.0,0.0,NaN,NaN,NaN,0.0,0.0,NaN,NaN,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021,2,22
1,Afghanistan,AFG,2021-02-23,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021,2,23
2,Afghanistan,AFG,2021-02-24,NaN,NaN,NaN,NaN,1367.0,NaN,NaN,NaN,34.0,"Johnson&Johnson, Oxford/AstraZeneca, Pfizer/Bi...",World Health Organization,https://covid19.who.int/,2021,2,24


### Import and preview PFE price data

In [47]:
pfe_prices_df = pd.read_csv('./data/PFE_historical_prices.csv')
pfe_prices_df['Date'] = pd.to_datetime(pfe_prices_df['Date'])
pfe_prices_df['year'] = pfe_prices_df.apply(lambda row: row.Date.year, axis=1)
pfe_prices_df['month'] = pfe_prices_df.apply(lambda row: row.Date.month, axis=1)
pfe_prices_df['day'] = pfe_prices_df.apply(lambda row: row.Date.day, axis=1)
pfe_prices_df.head(3)

,Date,Open,High,Low,Close,Adj Close,Volume,year,month,day
0,2020-12-02,40.470001,41.410000,40.299999,40.799999,38.354626,84347500,2020,12,2
1,2020-12-03,40.980000,41.029999,39.520000,40.090000,37.687180,68570100,2020,12,3
2,2020-12-04,39.889999,40.450001,39.889999,40.340000,37.922195,35368100,2020,12,4


### Iterate over vaccination data and upload to Firebase

In [51]:
all_countries = vacc_df['country'].unique()
all_years = [2020, 2021, 2022]
all_months = [month for month in range(1, 13)]
vacc_dict = {}

for country in all_countries:
    country_df = vacc_df.loc[vacc_df['country'] == country]
    country_dict = {}
    
    for year in all_years:
        year_df = vacc_df.loc[vacc_df['year'] == year]
        year_dict = {}
        
        for month in all_months:
            month_df = vacc_df.loc[vacc_df['month'] == month]
            month_dict = dict(month_df[['day', 'daily_vaccinations']].values)
            month_dict = {int(k):v for k,v in month_dict.items()}
            
            year_dict[month] = month_dict
        country_dict[year] = year_dict
    vacc_dict[country] = country_dict
    
url = 'https://dsci-551-3cb2b-default-rtdb.firebaseio.com/vaccination_data.json'
result = requests.put(url, data=json.dumps(vacc_dict))
print(result)

<Response [200]>


### Iterate over PFE price data and upload to Firebase

In [52]:
pfe_dict = {}

for year in all_years:
    year_df = pfe_prices_df.loc[pfe_prices_df['year'] == year]
    year_dict = {}

    for month in all_months:
        month_df = pfe_prices_df.loc[pfe_prices_df['month'] == month]
        month_dict = dict(month_df[['day', 'Close']].values)
        month_dict = {int(k):v for k,v in month_dict.items()}
        
        year_dict[month] = month_dict
    pfe_dict[year] = year_dict

url = 'https://dsci-551-3cb2b-default-rtdb.firebaseio.com/pfe_prices_data.json'
result = requests.put(url, data=json.dumps(pfe_dict))
print(result)

<Response [200]>


In [53]:
user_input = input("test:")
print(user_input)

test: here


here


In [2]:
cred = credentials.Certificate("./dsci-551-3cb2b-firebase-adminsdk-7ajia-c0dffd1d3c.json")
databaseURL = 'https://dsci-551-3cb2b-default-rtdb.firebaseio.com/'
default_app = firebase_admin.initialize_app(cred, {
    'databaseURL': databaseURL
})
ref = db.reference("/")

### EDFS Implementations

In [36]:
# rm
def remove_file(path):
    ref = db.reference(path)
    children = ref.get()
    for key, value in children.items():
        ref.child(key).set({})

# cat
def cat_file(path):
    ref = db.reference(path)
    print(ref.get())

# ls
def list_contents(path):
    ref = db.reference(path)
    contents = ref.get()
    for key, val in contents.items():
        print(key)
        
# mkdir
def make_directory(path):
    ref = db.reference(path)
    ref.set({
        "New Path":
        {
            "New_Path": True
        }
    })

# put (without partitions)
def put_file(file, path):
    ref = db.reference(path)
    query = ref.child("New Path").get()
    if len(query) == 1:
        ref.child("New Path").set({})
    with open(file, "r") as f:
        contents = json.load(f)
    ref.set(contents)

In [8]:
put_file("test.json", '/test/')

In [7]:
remove_file('/test/')

In [10]:
list_contents('/test/')

Person1
Person2
Person3


In [11]:
cat_file('/test/')

{'Person1': {'description': 'Dolor a sit illum vel corrupti est quia. Dolorem repudiandae molestiae qui ut. Repellendus in mollitia error repellat necessitatibus molestiae.', 'email': 'cale83@hotmail.com', 'name': 'Lorenza Nienow', 'phone': '1-786-677-7753'}, 'Person2': {'description': 'Amet veniam dolorum dolor sit voluptas. Voluptatem sit amet et provident. Culpa soluta nobis voluptatibus blanditiis animi veritatis officia.', 'email': 'dietrich.janet@gmail.com', 'name': 'Jack Koch PhD', 'phone': '947.729.3179'}, 'Person3': {'description': 'Eligendi voluptate in nostrum odit. Et voluptas mollitia suscipit. Optio quas ut voluptas.', 'email': 'patience67@gmail.com', 'name': 'Mr. Daryl Leffler Jr.', 'phone': '+1-551-867-7600'}}


In [39]:
make_directory('/test2/')

In [ ]:
remove_file('/test2/')

In [40]:
put_file("test.json", '/test2/')